In [1]:
import plotly.express as plt 
import pandas as pd
import os
import numpy as np
import sys 
sys.path.append('./modules/')
from mva import *

In [2]:
matrix = np.array([
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0.004,0.006,0.9,0.065,0.025],
    [0,0,1,0,0],
    [0,0,1,0,0]],np.dtype('d'))

In [9]:
mva = MVA(matrix,[5000,210,2.7,40,180],[StationType.Delay,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent],30)

def mvaToDataframe(matrix: np.ndarray)-> pd.DataFrame:
    vals = pd.DataFrame()
    vals["DELAY"] = matrix[0]
    vals["SWAP_IN"] = matrix[1]
    vals["CPU"] = matrix[2]
    vals["IO1"] = matrix[3]
    vals["IO2"] = matrix[4]
    vals["N"] = [i for i in range(0,len(matrix[0]))]
    return vals

mva()

In [16]:
#utilization = pd.read_csv("../../build/Scheduler/utilizations.csv",sep=";")
#throughputs = pd.read_csv("../../build/Scheduler/throughputs.csv", sep=";")
#meanWaits = pd.read_csv("../../build/Scheduler/meanWaits.csv",sep=";")
#meanClients = pd.read_csv("../../build/Scheduler/meanClients.csv", sep=";")

utilization = mvaToDataframe(mva.utilizations)
throughputs = mvaToDataframe(mva.throughputs)
meanWaits = mvaToDataframe(mva.meanwaits)
meanClients = mvaToDataframe(mva.meanclients)

visits = {}
visits["CPU"] = 250
visits["SWAP_IN"] = 2.5
visits["IO1"] = 16.25
visits["IO2"] = 6.25 

In [14]:


responseTimes = {}
responseTimes["CPU"] = meanWaits["CPU"]*visits["CPU"]
responseTimes["SWAP_IN"] = meanWaits["SWAP_IN"]*visits["SWAP_IN"]
responseTimes["IO1"] = meanWaits["IO1"]*visits["IO1"]
responseTimes["IO2"] =meanWaits["IO2"]*visits["IO2"]
responseTimes["N"] = meanWaits["N"]
print(responseTimes)


{'CPU': 0     1250000.0
1    11077500.0
2        2497.5
3      410000.0
4     8145000.0
Name: CPU, dtype: float64, 'SWAP_IN': 0    12500.00
1      525.00
2        6.75
3      100.00
4      450.00
Name: SWAP_IN, dtype: float64, 'IO1': 0    8.125000e+04
1    1.512113e+08
2    4.821863e+02
3    1.066650e+06
4    9.529942e+07
Name: IO1, dtype: float64, 'IO2': 0    3.125000e+04
1    1.221322e+10
2    5.176069e+02
3    1.641025e+07
4    6.597654e+09
Name: IO2, dtype: float64, 'N': 0    0
1    1
2    2
3    3
4    4
Name: N, dtype: int64}


In [12]:
def produce_graph_and_save(dataframe: pd.DataFrame, name:str, yaxis:str, log:bool = False):
    fig = plt.line(dataframe,y=["CPU","SWAP_IN","IO1","IO2"], x="N",title=name,log_y=log)
    fig.update_yaxes(title_text=yaxis)
    data = fig.to_image("png")
    f = open("./Images/{}.png".format(name),"+wb")
    f.write(data)
    f.close()
    fig.show()
pass

In [13]:
produce_graph_and_save(utilization,"Utilizations","B/T")

In [60]:
produce_graph_and_save(meanWaits,"meanWaits","ms",True)

In [61]:
produce_graph_and_save(meanClients,"meanClients","n")

In [62]:
produce_graph_and_save(throughputs,"throughputs","C/T")

In [63]:
produce_graph_and_save(responseTimes,"Response_Times","ms",True)

In [74]:
newmat = matrix.astype(np.double)
old = newmat.astype(np.double)
for i in range(50000):
    old = newmat.astype(np.double)
    newmat = old.astype(np.double) @ old.astype(np.double)
    newmat = newmat.round(10) #needed because overflow may occur
    pass
print(newmat)
pi = np.array([0]*len(matrix))
pi[0] = 1
pi = pi@newmat
pi.sum()

[[0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]]


1.0

In [80]:
app = [0]*len(pi)
for i in range(len(pi)):
    app[i] = pi[i]*newmat[i][i]
    pass
app

[1.312749418189456e-05,
 8.2046838636841e-05,
 0.8204683890858013,
 0.003466478944181895,
 0.0005127927414802562]